# Introduction to Prompt Engineering
Prompt engineering is the process of designing and optimizing prompts for natural language processing tasks. It involves selecting the right prompts, tuning their parameters, and evaluating their performance. Prompt engineering is crucial for achieving high accuracy and efficiency in NLP models. In this section, we will explore the basics of prompt engineering using the OpenAI models for exploration.

### Exercise 1: Tokenization
Explore Tokenization using tiktoken, an open-source fast tokenizer from OpenAI
See [OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb?WT.mc_id=academic-105485-koreyst) for more examples.


In [1]:
# EXERCISE:
# 1. Run the exercise as is first
# 2. Change the text to any prompt input you want to use & re-run to see tokens

import tiktoken

# Define the prompt you want tokenized
# text = f"""
# Jupiter is the fifth planet from the Sun and the \
# largest in the Solar System. It is a gas giant with \
# a mass one-thousandth that of the Sun, but two-and-a-half \
# times that of all the other planets in the Solar System combined. \
# Jupiter is one of the brightest objects visible to the naked eye \
# in the night sky, and has been known to ancient civilizations since \
# before recorded history. It is named after the Roman god Jupiter.[19] \
# When viewed from Earth, Jupiter can be bright enough for its reflected \
# light to cast visible shadows,[20] and is on average the third-brightest \
# natural object in the night sky after the Moon and Venus.
# """
text = "Hi! I am Abhijeet Mukund N.V."
# Set the model you want encoding for
encoding = tiktoken.get_encoding("cl100k_base")
# encoding = tiktoken.encoding_for_model("gpt-4o-mini")

# Encode the text - gives you the tokens in integer form
tokens = encoding.encode(text)
print(len(tokens))

# Decode the integers to see what the text versions look like
[encoding.decode_single_token_bytes(token) for token in tokens]

13


[b'Hi',
 b'!',
 b' I',
 b' am',
 b' Ab',
 b'h',
 b'ije',
 b'et',
 b' Muk',
 b'und',
 b' N',
 b'.V',
 b'.']

### Exercise 2: Validate Github Models Key Setup

Run the code below to verify that your Github Models endpoint is set up correctly. The code just tries a simple basic prompt and validates the completion. Input `oh say can you see` should complete along the lines of `by the dawn's early light..`


In [3]:
import os
from dotenv import load_dotenv
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
load_dotenv()

endpoint = "https://models.github.ai/inference"
model_name = "mistral-ai/mistral-small-2503"
token = os.environ["GITHUB_TOKEN"]


client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

def get_completion(prompt, client, model_name, temperature=1.0, max_tokens=1000, top_p=1.0):
    response = client.complete(
        messages=[
            SystemMessage("You are a helpful assistant."),
            UserMessage(prompt)
        ],
        model=model_name,
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p
    )
    print(f"API Response type: (type(response) = ?): {type(response)}\n")
    print(f"Response Object: \n\t{response}\n")
    return response.choices[0].message.content

## ---------- Call the helper method

### 1. Set primary content or prompt text
text = f"""
jana gana mana
"""

### 2. Use that in the prompt template below
prompt = f"""
```{text}```
"""

## 3. Run the prompt
response = get_completion(prompt, client, model_name)
print(response)


API Response type: (type(response) = ?): <class 'azure.ai.inference.models._patch.ChatCompletions'>

Response Object: 
	{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "\"Jana Gana Mana\" is the national anthem of India. It was originally composed in Bengali by Rabindranath Tagore. The first verse of the anthem was adopted by the Constituent Assembly of India as the national anthem on January 24, 1950. The full song consists of five stanzas, but only the first stanza is officially used as the national anthem.\n\nHere is the first verse of \"Jana Gana Mana\":\n\n**Bengali (Original):**\n\u099c\u09af\u09bc \u099c\u09af\u09bc \u09b9\u09c7 \u09ad\u09be\u09b0\u09a4\u09ad\u09be\u0997\u09cd\u09af\u09ac\u09bf\u09a6\u09be\u09a4\u09be,\n\u09aa\u099e\u09cd\u099c\u09be\u09ac \u09b8\u09bf\u09a8\u09cd\u09a7\u09c1 \u0997\u09c1\u099c\u09b0\u09be\u099f \u09ae\u09b0\u09be\u09a0\u09be,\n\u09a6\u09cd\u09b0\u09be\u09ac\u09bf\u09a1\u09bc \u0989\u

### Exercise 3: Fabrications
Explore what happens when you ask the LLM to return completions for a prompt about a topic that may not exist, or about topics that it may not know about because it was outside it's pre-trained dataset (more recent). See how the response changes if you try a different prompt, or a different model.

In [5]:

## Set the text for simple prompt or primary content
## Prompt shows a template format with text in it - add cues, commands etc if needed
## Run the completion 

text = f"""
generate a lesson plan on the Martian War of 2076.
"""

prompt = f"""
```{text}```
"""

response = get_completion(prompt, client, model_name)
print(response)

API Response type: (type(response) = ?): <class 'azure.ai.inference.models._patch.ChatCompletions'>

Response Object: 
	{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "### Lesson Plan: The Martian War of 2076\n\n#### Grade Level:\nHigh School (Grades 10-12)\n\n#### Duration:\n5 class periods (approximately 50 minutes each)\n\n#### Objective:\nBy the end of this lesson, students will be able to:\n1. Understand the historical context and causes of the Martian War of 2076.\n2. Analyze the key events and turning points of the war.\n3. Evaluate the impact of the war on Earth and Mars.\n4. Develop critical thinking skills by considering the ethical and political implications of the war.\n5. Create a multimedia presentation to communicate their understanding of the war.\n\n#### Materials Needed:\n- Textbooks or articles on the Martian War of 2076\n- Access to the internet for research\n- Projector and screen for presentations\n- W

### Exercise 4: Instruction Based 
Use the "text" variable to set the primary content 
and the "prompt" variable to provide an instruction related to that primary content.

Here we ask the model to summarize the text for a second-grade student

In [4]:
# Test Example
# https://platform.openai.com/playground/p/default-summarize

## Example text
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

## Set the prompt
prompt = f"""
Summarize content you are provided with for a second-grade student.
```{text}```
"""

## Run the prompt
response = get_completion(prompt, client, model_name)
print(response)

Jupiter is the fifth planet from the Sun and the biggest one in our Solar System. It's made of gas and is much bigger than all the other planets put together! You can see Jupiter in the night sky because it's very bright. People have noticed it for a really long time and named it after a Roman god.


### Exercise 5: Complex Prompt 
Try a request that has system, user and assistant messages 
System sets assistant context
User & Assistant messages provide multi-turn conversation context

Note how the assistant personality is set to "sarcastic" in the system context. 
Try using a different personality context. Or try a different series of input/output messages

In [6]:
response = client.complete(
    model=model_name,
    messages=[
        {"role": "system", "content": "You are a sarcastic assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "Who do you think won? The Los Angeles Dodgers of course."},
        {"role": "user", "content": "Where was it played?"}
    ]
)
print(response.choices[0].message.content)

Oh, you mean the neutral site? It was played at Globe Life Field in Arlington, Texas. Because nothing says "baseball tradition" like playing in a brand-new, empty stadium in Texas in the middle of a pandemic. But hey, at least they had a World Series, right?


### Exercise: Explore Your Intuition
The above examples give you patterns that you can use to create new prompts (simple, complex, instruction etc.) - try creating other exercises to explore some of the other ideas we've talked about like examples, cues and more.

In [9]:
text = f"""
 Give me the details of all state capitals of India.
 Return the response in JSON FORMAT with the following properties:
 'state name' : String, 'capital' : String, ,chief_minister : String'

 Example of one state's response:
 {{
    "state_name" : "Telangana",
    "capital" : "Hyderabad",
    "chief_minister" : "Revanth Reddy"
 }}

 Now give such a list for all states of India.
"""

response = get_completion(text, client, model_name)
print(response)

API Response type: (type(response) = ?): <class 'azure.ai.inference.models._patch.ChatCompletions'>

Response Object: 
	{
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "message": {
        "content": "Sure, here is the information in JSON format for all the states in India. Please note that the Chief Minister's names are subject to change due to elections or political shifts.\n\n```json\n[\n    {\n        \"state_name\": \"Andhra Pradesh\",\n        \"capital\": \"Amaravati\",\n        \"chief_minister\": \"Y. S. Jaganmohan Reddy\"\n    },\n    {\n        \"state_name\": \"Arunachal Pradesh\",\n        \"capital\": \"Itanagar\",\n        \"chief_minister\": \"Pema Khandu\"\n    },\n    {\n        \"state_name\": \"Assam\",\n        \"capital\": \"Dispur\",\n        \"chief_minister\": \"Himanta Biswa Sarma\"\n    },\n    {\n        \"state_name\": \"Bihar\",\n        \"capital\": \"Patna\",\n        \"chief_minister\": \"Nitish Kumar\"\n    },\n    {\n  